## <center>POKEMON DCGAN</center>

#### Importing all the required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, metrics, losses, optimizers, Model
import cv2
import os
# from google.colab.patches import cv2_imshow
tf.random.set_seed(45)

#### Model

In [2]:
class DCGANGenerator(Model):
    def __init__(self, latent_dim):
        super(DCGANGenerator, self).__init__()
        
        self.dense_1 = layers.Dense(units=latent_dim*2*2, input_dim=latent_dim)
        self.activation_1 = layers.Activation('relu')
        self.reshape_1 = layers.Reshape(target_shape=(2, 2, latent_dim))

        self.upsampling_2 = layers.UpSampling2D()
        self.conv_2 = layers.Conv2D(filters=512, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_2 = layers.BatchNormalization(momentum=0.7)
        self.activation_2 = layers.Activation('relu')

        self.upsampling_3 = layers.UpSampling2D()
        self.conv_3 = layers.Conv2D(filters=512, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_3 = layers.BatchNormalization(momentum=0.7)
        self.activation_3 = layers.Activation('relu')

        self.upsampling_4 = layers.UpSampling2D()
        self.conv_4 = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_4 = layers.BatchNormalization(momentum=0.7)
        self.activation_4 = layers.Activation('relu')

        self.upsampling_5 = layers.UpSampling2D()
        self.conv_5 = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_5 = layers.BatchNormalization(momentum=0.7)
        self.activation_5 = layers.Activation('relu')

        self.upsampling_6 = layers.UpSampling2D()
        self.conv_6 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_6 = layers.BatchNormalization(momentum=0.8)
        self.activation_6 = layers.Activation('relu')

        self.upsampling_7 = layers.UpSampling2D()
        self.conv_7 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_7 = layers.BatchNormalization(momentum=0.7)
        self.activation_7 = layers.Activation('relu')

        self.conv_8 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_8 = layers.BatchNormalization(momentum=0.7)
        self.activation_8 = layers.Activation('relu')

        self.conv_9 = layers.Conv2D(filters=3, kernel_size=(3, 3), strides=1, padding='same')
        self.activation_9 = layers.Activation('tanh')
    
    def call(self, x):

        x = self.dense_1(x)
        x = self.activation_1(x)
        x = self.reshape_1(x)

        x = self.upsampling_2(x)
        x = self.conv_2(x)
        x = self.norm_2(x)
        x = self.activation_2(x)

        x = self.upsampling_3(x)
        x = self.conv_3(x)
        x = self.norm_3(x)
        x = self.activation_3(x)

        x = self.upsampling_4(x)
        x = self.conv_4(x)
        x = self.norm_4(x)
        x = self.activation_4(x)

        x = self.upsampling_5(x)
        x = self.conv_5(x)
        x = self.norm_5(x)
        x = self.activation_5(x)

        x = self.upsampling_6(x)
        x = self.conv_6(x)
        x = self.norm_6(x)
        x = self.activation_6(x)

        x = self.upsampling_7(x)
        x = self.conv_7(x)
        x = self.norm_7(x)
        x = self.activation_7(x)

#         x = self.upsampling_8(x)
        x = self.conv_8(x)
        x = self.norm_8(x)
        x = self.activation_8(x)

        x = self.conv_9(x)
        out = self.activation_9(x)

        return out

In [3]:
class DCGANDiscriminator(Model):
    def __init__(self, image_shape):
        super(DCGANDiscriminator, self).__init__()

        self.conv_1 = layers.Conv2D(filters=32, kernel_size=(3, 3), strides=2, padding='same')
        self.activation_1 = layers.LeakyReLU(alpha=0.2)
        self.dropout_1 = layers.Dropout(rate=0.25)

        self.conv_2 = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=2, padding='same')
        self.zeropad_2 = layers.ZeroPadding2D(padding=((0, 1), (0, 1)))
        self.norm_2 = layers.BatchNormalization(momentum=0.8)
        self.activation_2 = layers.LeakyReLU(alpha=0.2)
        self.dropout_2 = layers.Dropout(rate=0.25)

        self.conv_3 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=2, padding='same')
        self.norm_3 = layers.BatchNormalization(momentum=0.8)
        self.activation_3 = layers.LeakyReLU(alpha=0.1)
        self.dropout_3 = layers.Dropout(rate=0.25)

        self.conv_4 = layers.Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same')
        self.norm_4 = layers.BatchNormalization(momentum=0.8)
        self.activation_4 = layers.LeakyReLU(alpha=0.1)
        self.dropout_4 = layers.Dropout(rate=0.25)

        self.flatten = layers.Flatten()
        self.dense_5 = layers.Dense(units=1)
        self.activation_5 = layers.Activation('sigmoid')
    
    def call(self, x):
        # print(X.shape)
        x = self.conv_1(x)
        x = self.activation_1(x)
        x = self.dropout_1(x)
        # print(x.shape)

        x = self.conv_2(x)
        x = self.zeropad_2(x)
        x = self.norm_2(x)
        x = self.activation_2(x)
        x = self.dropout_2(x)
        # print(x.shape)

        x = self.conv_3(x)
        x = self.norm_3(x)
        x = self.activation_3(x)
        x = self.dropout_3(x)
        # print(x.shape)

        x = self.conv_4(x)
        x = self.norm_4(x)
        x = self.activation_4(x)
        x = self.dropout_4(x)
        # print(x.shape)

        x = self.flatten(x)
        # print(x.shape)

        x = self.dense_5(x)
        # print(x.shape)
        out = self.activation_5(x)

        return out

In [4]:
def train_step(gen_model, dis_model, vgg16_model, loss_method, loss_method_vgg, optimizer_method_gen, optimizer_method_dis, batch_size, latent_dim, X):
    latent_vector = tf.random.normal(shape=(batch_size, latent_dim))
    valid_labels = tf.ones(shape=(batch_size, 1))
    fake_labels = tf.zeros(shape=(batch_size, 1))
    # X = tf.expand_dims(X, axis=3)
#     print(X.shape)
    with tf.GradientTape() as tape:
        gen_out = gen_model(latent_vector, training=True)
#         print(gen_out.shape)
        real_out = dis_model(X, training=True)
        fake_out = dis_model(gen_out, training=True)
        real_loss = loss_method(valid_labels, real_out)
        fake_loss = loss_method(fake_labels, fake_out)
        dis_loss = tf.multiply(tf.add(real_loss, fake_loss), 1.0)
    variables = gen_model.trainable_variables + dis_model.trainable_variables
    gradients = tape.gradient(dis_loss, variables)
    optimizer_method_dis.apply_gradients(zip(gradients, variables))

    with tf.GradientTape() as tape:
        gen_out = gen_model(latent_vector, training=True)
        fake_out = dis_model(gen_out, training=True)
        gen_loss = loss_method(valid_labels, fake_out)
    variables = gen_model.trainable_variables
    gradients = tape.gradient(gen_loss, variables)
    optimizer_method_gen.apply_gradients(zip(gradients, variables))

    # with tf.GradientTape() as tape:
    #     gen_out = gen_model(latent_vector, training=True)
    #     fake_out = dis_model(gen_out, training=True)
    #     gen_feat = vgg16_model(gen_out)
    #     real_feat = vgg16_model(X)
    #     vgg_loss = loss_method_vgg(real_feat, gen_feat)
    # variables = gen_model.trainable_variables
    # gradients = tape.gradient(vgg_loss, variables)
    # optimizer_method.apply_gradients(zip(gradients, variables))

    return gen_loss, dis_loss

In [5]:
def generate_images(epoch, latent_dim, gen_model):
    r, c = 5, 5
    latent_vector = tf.random.normal(shape=(r*c, latent_dim))
    gen_imgs = gen_model(latent_vector, training=True)
    gen_imgs = gen_imgs * 0.5 + 0.5
    fig, axis = plt.subplots(r, c, figsize=(15, 15))
    idx = 0
    for i in range(r):
        for j in range(c):
            axis[i, j].imshow(gen_imgs[idx])
            # axis[i, j].imshow(gen_imgs[idx, :, :, 0], cmap='gray')
            axis[i, j].axis('off')
            idx+=1
    fig.savefig('data/dcgan_color/output/{0}.png'.format(epoch))
    plt.close()
# generate_images(epoch, latent_dim, gen_model)

#### Dataset Prepration

In [6]:
# This list contains all the image data
data = []
image_shape = (128, 128, 3)
# image_shape = (128, 128)
latent_dim = 128
batch_size = 16
data_dir = 'data/dcgan_color/pokemon/'
ckpt_dir = 'data/dcgan_color/checkpoint/'
# train_batch = tf.keras.preprocessing.image_dataset_from_directory(directory=data_dir, seed=45, image_size=(120, 120), batch_size=batch_size)
# print(train_batch.shape)
for file_name in os.listdir(path=data_dir):
    img_path = os.path.join(data_dir, file_name)
    img = cv2.imread(img_path, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if img.shape[0]<=120 or img.shape[1]<=120:
        img = np.pad(img, pad_width=((4, 4), (4, 4), (0, 0)), mode='constant')
    else:
        img = cv2.resize(img, (128, 128))
#     cv2.imshow('Image', img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    data.append(img)
data = np.array(data)
print(data.shape)

(1628, 128, 128, 3)


In [7]:
# Here we are creating training batch
train_batch = tf.data.Dataset.from_tensor_slices(data).shuffle(1000).batch(batch_size, drop_remainder=True)

In [8]:
gen_model = DCGANGenerator(latent_dim=latent_dim)
dis_model = DCGANDiscriminator(image_shape=image_shape)
# vgg16_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
# vgg16_model_input = vgg16_model.input
# vgg16_model_output = vgg16_model.layers[-3].output
# vgg16_model = Model(inputs=vgg16_model_input, outputs=vgg16_model_output)
vgg16_model = None

In [9]:
loss_method = losses.BinaryCrossentropy()
loss_method_vgg = losses.MeanSquaredError()
optimizer_method_gen = optimizers.Adam(learning_rate=0.001)
optimizer_method_dis = optimizers.Adam(learning_rate=0.0001)
ckpt = tf.train.Checkpoint(step=tf.Variable(0), optimizer_g=optimizer_method_gen, optimizer_d=optimizer_method_dis, g_model=gen_model, d_model=dis_model)
manager = tf.train.CheckpointManager(checkpoint=ckpt, directory=ckpt_dir, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print('Restored from last checkpoint : {0}'.format(int(ckpt.step)))
START_EPOCH = int(ckpt.step)
EPOCHS = 10000

In [10]:
for epoch in range(START_EPOCH, EPOCHS):
    gen_loss = metrics.Mean()
    dis_loss = metrics.Mean()
    for X in train_batch:
        X = (tf.cast(X, dtype='float32')-127.5)/127.5
#         X = tf.cast(X, dtype='float32')/255.0
        g_loss, d_loss = train_step(gen_model, dis_model, vgg16_model, loss_method, loss_method_vgg, optimizer_method_gen, optimizer_method_dis, batch_size, latent_dim, X)
        gen_loss.update_state(g_loss)
        dis_loss.update_state(d_loss)
    ckpt.step.assign_add(1)
    manager.save()
    print('Epoch : {0}\tGen Loss : {1}\tDis Loss : {2}'.format(epoch, gen_loss.result(), dis_loss.result()))
    if epoch%2 == 0:
        generate_images(epoch, latent_dim, gen_model)

Epoch : 0	Gen Loss : 0.8680251240730286	Dis Loss : 2.0738515853881836
Epoch : 1	Gen Loss : 0.8651326894760132	Dis Loss : 2.2525596618652344
Epoch : 2	Gen Loss : 0.8116071820259094	Dis Loss : 2.294450283050537
Epoch : 3	Gen Loss : 1.2024956941604614	Dis Loss : 1.771863579750061
Epoch : 4	Gen Loss : 1.2277148962020874	Dis Loss : 1.8075473308563232
Epoch : 5	Gen Loss : 2.1321544647216797	Dis Loss : 1.0573073625564575
Epoch : 6	Gen Loss : 1.5488337278366089	Dis Loss : 1.4291937351226807
Epoch : 7	Gen Loss : 2.4057488441467285	Dis Loss : 0.9505031108856201
Epoch : 8	Gen Loss : 2.121460199356079	Dis Loss : 1.3485264778137207
Epoch : 9	Gen Loss : 2.5372538566589355	Dis Loss : 0.9479221105575562
Epoch : 10	Gen Loss : 2.0550472736358643	Dis Loss : 1.207485556602478
Epoch : 11	Gen Loss : 2.0531675815582275	Dis Loss : 1.3342304229736328
Epoch : 12	Gen Loss : 2.0888590812683105	Dis Loss : 1.3659300804138184
Epoch : 13	Gen Loss : 2.953885316848755	Dis Loss : 0.5463862419128418
Epoch : 14	Gen Loss :

Epoch : 117	Gen Loss : 2.158385992050171	Dis Loss : 1.5138702392578125
Epoch : 118	Gen Loss : 2.3027853965759277	Dis Loss : 1.2655025720596313
Epoch : 119	Gen Loss : 2.095308542251587	Dis Loss : 1.5480506420135498
Epoch : 120	Gen Loss : 2.059492588043213	Dis Loss : 1.8011982440948486
Epoch : 121	Gen Loss : 2.3829545974731445	Dis Loss : 0.9770318269729614
Epoch : 122	Gen Loss : 1.7090164422988892	Dis Loss : 1.787089467048645
Epoch : 123	Gen Loss : 2.0724036693573	Dis Loss : 1.465407133102417
Epoch : 124	Gen Loss : 2.2435431480407715	Dis Loss : 1.4745781421661377
Epoch : 125	Gen Loss : 1.7130746841430664	Dis Loss : 1.882589340209961
Epoch : 126	Gen Loss : 1.6684080362319946	Dis Loss : 1.4617208242416382
Epoch : 127	Gen Loss : 1.7590795755386353	Dis Loss : 1.688011884689331
Epoch : 128	Gen Loss : 1.6801587343215942	Dis Loss : 1.6291491985321045
Epoch : 129	Gen Loss : 1.950722098350525	Dis Loss : 1.2701421976089478
Epoch : 130	Gen Loss : 1.8821172714233398	Dis Loss : 1.438157081604004
Epoc

Epoch : 232	Gen Loss : 3.158890724182129	Dis Loss : 0.6790491938591003
Epoch : 233	Gen Loss : 2.732924699783325	Dis Loss : 0.7559148073196411
Epoch : 234	Gen Loss : 2.9851884841918945	Dis Loss : 0.6458168625831604
Epoch : 235	Gen Loss : 3.1803770065307617	Dis Loss : 0.6204944849014282
Epoch : 236	Gen Loss : 3.0835020542144775	Dis Loss : 0.7725185751914978
Epoch : 237	Gen Loss : 3.259591817855835	Dis Loss : 0.6302311420440674
Epoch : 238	Gen Loss : 2.9334909915924072	Dis Loss : 0.8037496209144592
Epoch : 239	Gen Loss : 3.202584743499756	Dis Loss : 0.5236107110977173
Epoch : 240	Gen Loss : 3.2571029663085938	Dis Loss : 0.5138700008392334
Epoch : 241	Gen Loss : 3.1658129692077637	Dis Loss : 0.7352123260498047
Epoch : 242	Gen Loss : 3.1052474975585938	Dis Loss : 0.9195300340652466
Epoch : 243	Gen Loss : 3.051734209060669	Dis Loss : 0.903391420841217
Epoch : 244	Gen Loss : 2.9509785175323486	Dis Loss : 0.6621267199516296
Epoch : 245	Gen Loss : 3.017974615097046	Dis Loss : 0.5669730305671692

Epoch : 347	Gen Loss : 4.326124668121338	Dis Loss : 0.4538605511188507
Epoch : 348	Gen Loss : 4.180126190185547	Dis Loss : 0.3447507321834564
Epoch : 349	Gen Loss : 4.593588829040527	Dis Loss : 0.3980979025363922
Epoch : 350	Gen Loss : 4.296279430389404	Dis Loss : 0.5127062201499939
Epoch : 351	Gen Loss : 4.603015899658203	Dis Loss : 0.2565349340438843
Epoch : 352	Gen Loss : 4.751267433166504	Dis Loss : 0.1760808229446411
Epoch : 353	Gen Loss : 4.469106674194336	Dis Loss : 0.4868205189704895
Epoch : 354	Gen Loss : 3.9031553268432617	Dis Loss : 0.35889649391174316
Epoch : 355	Gen Loss : 4.1024298667907715	Dis Loss : 0.3536819517612457
Epoch : 356	Gen Loss : 4.2520904541015625	Dis Loss : 0.41975483298301697
Epoch : 357	Gen Loss : 4.05103063583374	Dis Loss : 0.4021901488304138
Epoch : 358	Gen Loss : 3.996171712875366	Dis Loss : 0.4625715911388397
Epoch : 359	Gen Loss : 4.242532730102539	Dis Loss : 0.40358594059944153
Epoch : 360	Gen Loss : 3.934079885482788	Dis Loss : 0.41228187084198
Epo

Epoch : 462	Gen Loss : 5.013097286224365	Dis Loss : 0.3336033225059509
Epoch : 463	Gen Loss : 4.875233173370361	Dis Loss : 0.33267539739608765
Epoch : 464	Gen Loss : 4.869129657745361	Dis Loss : 0.3399372696876526
Epoch : 465	Gen Loss : 4.629022121429443	Dis Loss : 0.29715684056282043
Epoch : 466	Gen Loss : 4.770272254943848	Dis Loss : 0.27356788516044617
Epoch : 467	Gen Loss : 4.708646774291992	Dis Loss : 0.3402584195137024
Epoch : 468	Gen Loss : 4.6826677322387695	Dis Loss : 0.310535192489624
Epoch : 469	Gen Loss : 4.723169803619385	Dis Loss : 0.28139472007751465
Epoch : 470	Gen Loss : 4.826076984405518	Dis Loss : 0.2754647731781006
Epoch : 471	Gen Loss : 5.130561351776123	Dis Loss : 0.22092439234256744
Epoch : 472	Gen Loss : 5.437763690948486	Dis Loss : 0.2606185972690582
Epoch : 473	Gen Loss : 5.24452543258667	Dis Loss : 0.27226775884628296
Epoch : 474	Gen Loss : 4.941282272338867	Dis Loss : 0.2765052914619446
Epoch : 475	Gen Loss : 5.473836898803711	Dis Loss : 0.22277554869651794


...

Epoch : 6363	Gen Loss : 13.002368927001953	Dis Loss : 0.05454721674323082
Epoch : 6364	Gen Loss : 14.222506523132324	Dis Loss : 0.011954249814152718
Epoch : 6365	Gen Loss : 13.595041275024414	Dis Loss : 0.0170870553702116
Epoch : 6366	Gen Loss : 13.390811920166016	Dis Loss : 0.011071796528995037
Epoch : 6367	Gen Loss : 12.185352325439453	Dis Loss : 0.0497700572013855
Epoch : 6368	Gen Loss : 12.957032203674316	Dis Loss : 0.04894811660051346
Epoch : 6369	Gen Loss : 13.060212135314941	Dis Loss : 0.030945708975195885
Epoch : 6370	Gen Loss : 13.7828369140625	Dis Loss : 0.027039244771003723
Epoch : 6371	Gen Loss : 13.045866012573242	Dis Loss : 0.06763281673192978
Epoch : 6372	Gen Loss : 13.93992805480957	Dis Loss : 0.026287730783224106
Epoch : 6373	Gen Loss : 12.212303161621094	Dis Loss : 0.05253790318965912
Epoch : 6374	Gen Loss : 11.718145370483398	Dis Loss : 0.06406611204147339
Epoch : 6375	Gen Loss : 11.972757339477539	Dis Loss : 0.07629744708538055
Epoch : 6376	Gen Loss : 10.83901119232

Epoch : 6474	Gen Loss : 11.967506408691406	Dis Loss : 0.07967229187488556
Epoch : 6475	Gen Loss : 11.073301315307617	Dis Loss : 0.049625419080257416
Epoch : 6476	Gen Loss : 13.181215286254883	Dis Loss : 0.059261128306388855
Epoch : 6477	Gen Loss : 11.860883712768555	Dis Loss : 0.07466846704483032
Epoch : 6478	Gen Loss : 12.219388008117676	Dis Loss : 0.038355063647031784
Epoch : 6479	Gen Loss : 11.497215270996094	Dis Loss : 0.05017891526222229
Epoch : 6480	Gen Loss : 10.984840393066406	Dis Loss : 0.0862705335021019
Epoch : 6481	Gen Loss : 10.273154258728027	Dis Loss : 0.10361611098051071
Epoch : 6482	Gen Loss : 11.157241821289062	Dis Loss : 0.04708614945411682
Epoch : 6483	Gen Loss : 11.618534088134766	Dis Loss : 0.042848698794841766
Epoch : 6484	Gen Loss : 12.664088249206543	Dis Loss : 0.06290166079998016
Epoch : 6485	Gen Loss : 12.545035362243652	Dis Loss : 0.0671314224600792
Epoch : 6486	Gen Loss : 12.696870803833008	Dis Loss : 0.029958903789520264
Epoch : 6487	Gen Loss : 11.47592544

Epoch : 6585	Gen Loss : 12.073811531066895	Dis Loss : 0.03686387836933136
Epoch : 6586	Gen Loss : 12.555574417114258	Dis Loss : 0.09576817601919174
Epoch : 6587	Gen Loss : 11.698492050170898	Dis Loss : 0.036229878664016724
Epoch : 6588	Gen Loss : 12.589126586914062	Dis Loss : 0.0545327290892601
Epoch : 6589	Gen Loss : 13.313570976257324	Dis Loss : 0.1254616677761078
Epoch : 6590	Gen Loss : 12.820718765258789	Dis Loss : 0.021968437358736992
Epoch : 6591	Gen Loss : 12.309924125671387	Dis Loss : 0.053843654692173004
Epoch : 6592	Gen Loss : 12.837656021118164	Dis Loss : 0.046167127788066864
Epoch : 6593	Gen Loss : 12.756036758422852	Dis Loss : 0.048231661319732666
Epoch : 6594	Gen Loss : 12.737665176391602	Dis Loss : 0.05291607230901718
Epoch : 6595	Gen Loss : 13.297974586486816	Dis Loss : 0.07428038120269775
Epoch : 6596	Gen Loss : 12.459293365478516	Dis Loss : 0.037222541868686676
Epoch : 6597	Gen Loss : 11.869485855102539	Dis Loss : 0.12507203221321106
Epoch : 6598	Gen Loss : 13.1763820

Epoch : 6696	Gen Loss : 12.613119125366211	Dis Loss : 0.0474945493042469
Epoch : 6697	Gen Loss : 12.327497482299805	Dis Loss : 0.041256580501794815
Epoch : 6698	Gen Loss : 11.754409790039062	Dis Loss : 0.1590145081281662
Epoch : 6699	Gen Loss : 13.017823219299316	Dis Loss : 0.029841968789696693
Epoch : 6700	Gen Loss : 11.195683479309082	Dis Loss : 0.04680369794368744
Epoch : 6701	Gen Loss : 12.533514976501465	Dis Loss : 0.08642158657312393


KeyboardInterrupt: 